In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
trashpick = pd.read_csv(r"C:\Users\ekims\Documents\NSS\DA14\Projects\trash-pickups-v2-ekimshaf\data\trash_hauler_report_with_lat_lng.csv")
print(trashpick.shape)
trashpick.head(20)

(20226, 13)


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140
6,25325,11/1/2017,Damage to Property,Trash/emptied Wednesday & now metal black-mail...,4721 Chalmers Dr,37215.0,RED RIVER,3303,34.0,1726300.965,637078.4022,-86.819640,36.080729
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203.0,METRO,9208,21.0,1731459.367,666013.6012,-86.802988,36.160330
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728
9,25331,11/1/2017,Trash - Curbside/Alley Missed Pickup,"Trash missed Tuesday again, ALLEY",1206 Ireland St,37208.0,METRO,9208,19.0,1733029.474,668762.6409,-86.797745,36.167916


In [6]:
## filter new datafram to include only instances for provider "Red River"
red_river = trashpick[trashpick['Trash Hauler'].isin(['RED RIVER'])]
print(red_river.shape)
red_river.head(10)

(14395, 13)


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140
6,25325,11/1/2017,Damage to Property,Trash/emptied Wednesday & now metal black-mail...,4721 Chalmers Dr,37215.0,RED RIVER,3303,34.0,1726300.965,637078.4022,-86.819640,36.080729
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728
10,25341,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed trash pickup - resident has at curb for...,3113 HYDES FERRY RD,37218.0,RED RIVER,4204,2.0,1721577.768,676018.3999,-86.836750,36.187584
11,25350,11/1/2017,Trash Collection Complaint,Missed- 4th week in a row.,3210 Whites Creek Pike,37207.0,RED RIVER,3201,3.0,1726432.966,691919.2003,-86.820750,36.231374
14,25370,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,6543 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1706941.584,659614.1836,-86.885842,36.142170


In [31]:
## filter new dataframe for only values where description includes the word "miss"
## letter case was ignored, used Description column as Request column had more than one description type associated with missed p/u
rr_missed = red_river[red_river['Request'].str.contains('Trash - Curbside/Alley Missed Pickup', case=False, na=False)]
print(rr_missed.shape)
rr_missed.head(5)


(10715, 13)


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728
10,25341,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed trash pickup - resident has at curb for...,3113 HYDES FERRY RD,37218.0,RED RIVER,4204,2.0,1721577.768,676018.3999,-86.836750,36.187584


In [28]:
## counting instances where addresses appear more than once from missed p/u df

address_counts = rr_missed['Incident Address'].value_counts()
rpt_miss = address_counts[address_counts >= 2]

print(address_counts.shape)
print(rpt_miss.shape)
rpt_miss.head(20)

(8190,)
(1551,)


Incident Address
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
5135 Hickory Hollow Pkwy                                      16
802 Crescent Rd, Nashville, TN 37205, United States           16
607 Estes Rd, Nashville, TN 37215, United States              15
3710 N NATCHEZ CT                                             14
6007 Obrien Ave, Nashville, TN 37209, United States           14
320 Old Hickory Blvd, Nashville, TN 37221, United States      12
111 Barton Ln, Nashville, TN 37214, United States             11
1537 Harding Pl, Nashville, TN 37215, United States           11
3929 Stewarts Ln                                              10
208 Flint Ridge Ct, Nashville, TN 37189, United States         9
2731 Murfreesboro Pike                                         9
115 Barton Ln, Nashville, TN 37214, United States              9
1916 S Hamilton Rd                                             9
1584 Bell Rd                                                   9
14885 ol

In [29]:
## summing the total number of missed incidents with addresses where p/u miss is >= 2
missed_sum = sum(rpt_miss)
missed_sum


4075

In [33]:
## subtracting first instance from counts as it is not subject to fine
## calculated by subtracting count of missed p/u from sum of instances
penalty = missed_sum - rpt_miss.count()
penalty

2524

In [34]:
## calculating total fine, formatting to make it currency
tot_fine = "${:,.2f}".format(penalty * 200)
tot_fine


'$504,800.00'

Instruction on checking work:

1. Look at each calculation separately. How many incidents of missed pickups are there? Multiply that times the fee amount. That will tell you the max amount that the fine could be. 2. Look at the count of all house, then the count of all houses with more than one pickup. What percentage is that. If you take your total for above and reduce it by that percentage, do they line up?

In [38]:
tot_missed = red_river[red_river['Request'].str.contains('Trash - Curbside/Alley Missed Pickup', case=False, na=False)]




Request Number      15028
Date Opened         15028
Request             15028
Description         14999
Incident Address    15020
Zip Code            14972
Trash Hauler        14422
Trash Route         14405
Council District    14991
State Plan X        15007
State Plan Y        15007
LONGITUDE           15007
LATITUDE            15007
dtype: int64